In [ ]:
# jupyter nbconvert --to python test_software_dev_version.ipynb test_software_dev_version.py
%reset -f
import importlib
import numpy as np
import matplotlib.pyplot as plt

import sys

import Frog
import Frog.toolbox as toolbox
import Frog.frog_data_analysis as frog_data_analysis
# importlib.reload(frog_data_analysis)

import pickle
import time
from scipy.optimize import curve_fit


# Load the results

In [ ]:
# The directory where is located the FROG results:
directory = '/home/glebreton/Software/Frog/Doc/Tutorial_files/Beta_quadrupole'
name_result = 'L_moleculetype_result.p'

GP, L_moleculetype_result = frog_data_analysis.load_result(directory, name_result=name_result, what_to_print=['general info', 'diagram info'])

# Overview of the available values

In [ ]:
print('There are : ' + str(len(L_moleculetype_result[0].L_molecule)) + ' molecules for this MT.')
n = 0 # the molecule number
my_molecule = L_moleculetype_result[0].L_molecule[n] 
print('Here are the available attribute for this molecule:', my_molecule.L_attribute )

In [ ]:
print(my_molecule.mean_position)

# Electrostatic field

In [ ]:
electrostatic_on_fly = my_molecule.electric_field_on_fly
electrostatic_PE = my_molecule.electric_field_PE

diff = 0 
for frame in range(2):
    for obs in range(12):
        diff += (electrostatic_PE[frame][0][obs] + electrostatic_PE[frame][1][obs] - electrostatic_on_fly[frame][obs])**2

print('Should be zero:', diff)

In [ ]:
print('Dimension of my_molecule.electric_field_on_fly', electrostatic_on_fly.shape)

print('Laboratory electric field: EX, EY, EZ', electrostatic_on_fly[0][:3])
print('Laboratory electric field derivative in EX/dX, EY/dX, EZ/dX, EX/dY, EY/dY, EZ/dY, EX/dZ, EY/dZ, EZ/dZ', electrostatic_on_fly[0][3:])
print('Molecular electric field in Ex, Ey, Ez', electrostatic_on_fly[1][:3])
print('Molecular electric field derivative in Ex/dx, Ey/dx, Ez/dx, Ex/dy, Ey/dy, Ez/dy, Ex/dz, Ey/dz, Ez/dz', electrostatic_on_fly[1][3:])



In [ ]:
print('Dimension of my_molecule.electric_field_on_fly', electrostatic_PE.shape)

print('Laboratory Direct electric field: EX, EY, EZ', electrostatic_PE[0][0][:3])
print('Laboratory Direct  electric field derivative in EX/dX, EY/dX, EZ/dX, EX/dY, EY/dY, EZ/dY, EX/dZ, EY/dZ, EZ/dZ', electrostatic_PE[0][0][3:])

print('Laboratory Long-range electric field: EX, EY, EZ', electrostatic_PE[0][1][:3])
print('Laboratory Long-range electric field derivative in EX/dX, EY/dX, EZ/dX, EX/dY, EY/dY, EZ/dY, EX/dZ, EY/dZ, EZ/dZ', electrostatic_PE[0][1][3:])


# First hyperpolarizability 

In [ ]:
frequency_of_interest = '0_0'
frequency_of_interest = '0_05686'
frame = 'chi' # or 'beta'

print('Dipole-Dipole')
print(getattr(my_molecule, frame + '_' + frequency_of_interest))
print('Dipole-Quadrupole')
print(-getattr(my_molecule, frame + '_dq_' + frequency_of_interest))
print('Quadrupole-Dipole')
print(-getattr(my_molecule, frame + '_qd_' + frequency_of_interest))


# Check diagram encoding: 

In [ ]:
frequency_of_interest = '0_0'
frequency_of_interest = '0_05686'
frame = 'chi' # or 'beta'
i, j, k, l = 1, 1, 1, 1 

In [ ]:
# 'Dipole-Dipole'
MT_name = 'Water_TIP4P2005'
name_diagram = frame + '_' + frequency_of_interest
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)
K = i*9 + j*3 + k
plt.figure(1, figsize=(10, 7))
plt.plot(my_diagram.axis_observable.value, my_diagram.value[0][K], label='tensor element number '  + str(K), lw=3)
plt.xlabel(my_diagram.axis_observable.unit.print_unit(), fontsize=25)
plt.ylabel(my_diagram.unit.print_unit(), fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

print('Dipole-Dipole')
individual_value = getattr(my_molecule, frame + '_' + frequency_of_interest)[i][j][k]
average_value = my_diagram.mean[i][j][k]
print('individual value:', individual_value, 'average value:', average_value)
print('This should also match the distribution plotted!')

In [ ]:
# 'Dipole-Quadrupole'
MT_name = 'Water_TIP4P2005'
name_diagram = frame + '_dq_' + frequency_of_interest
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)
K = i*27 + j*9 + k*3 + l
plt.figure(1, figsize=(10, 7))
plt.plot(-my_diagram.axis_observable.value, my_diagram.value[0][K], label='tensor element number '  + str(K), lw=3)
plt.xlabel(my_diagram.axis_observable.unit.print_unit(), fontsize=25)
plt.ylabel(my_diagram.unit.print_unit(), fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

print('Dipole-Quadrupole')
individual_value = getattr(my_molecule, frame + '_dq_' + frequency_of_interest)[i][j][k][l]
average_value = my_diagram.mean[i][j][k][l]
print('individual value:', individual_value, 'average value:', average_value)
print('This should also match the distribution plotted!')

In [ ]:
# 'Dipole-Quadrupole'
MT_name = 'Water_TIP4P2005'
name_diagram = frame + '_qd_' + frequency_of_interest
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)
K = i*27 + j*9 + k*3 + l
plt.figure(1, figsize=(10, 7))
plt.plot(-my_diagram.axis_observable.value, my_diagram.value[0][K], label='tensor element number '  + str(K), lw=3)
plt.xlabel(my_diagram.axis_observable.unit.print_unit(), fontsize=25)
plt.ylabel(my_diagram.unit.print_unit(), fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

print('Quadrupole-Dipole')
individual_value = getattr(my_molecule, frame + '_qd_' + frequency_of_interest)[i][j][k][l]
average_value = my_diagram.mean[i][j][k][l]
print('individual value:', individual_value, 'average value:', average_value)
print('This should also match the distribution plotted!')